In [ ]:
!pip install google-cloud-aiplatform

In [ ]:
import time
import vertexai

from vertexai.batch_prediction import BatchPredictionJob
print("Code is correct")

Code is correct


In [ ]:
!gcloud auth login --no-launch-browser

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=V08BQfSf5iJ44WFVAXfk7DNEA8YmFr&prompt=consent&token_usage=remote&access_type=offline&code_challenge=CLCiaa7pawaZWJ50B1CYX_ZtJ1KCb0yNpLmDJ4kxIrs&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab_5qlm0MkMST34MXXtkgCuvazb14zG1tkvRXb2F6-VLHGPi5trSW4tTMV_hl7H3AYsnzg

You are now logged in as [zsdaniel@usc.edu].
Your current project is [N

In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=WlQGFfmCxRMC9bl2K3bPLFZoFhCwLo&prompt=consent&token_usage=remote&access_type=offline&code_challenge=PM8oCgkEHJuBCC9heKizYlXGIOzRXbaSX7RKGQBLFp4&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab_5qlm7wACLWGxLRJWvoIiynsEehBDfChFzmae-F_1kyIJcJ6pBcSe7lagsL8d61AF2hQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
!gcloud config set project csci-535-project


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].


In [ ]:
!gcloud auth application-default set-quota-project csci-535-project


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "csci-535-project" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [ ]:
from google.cloud import storage
PROJECT_ID = "csci-535-project"

In [ ]:
import vertexai

In [ ]:
import pandas as pd

# Load dataset CSVs
train_df = pd.read_csv("data/train_sent_emo.csv")
dev_df = pd.read_csv("data/dev_sent_emo.csv")
test_df = pd.read_csv("data/test_sent_emo.csv")

# Show first few rows
# train_df.head()

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(
    vertexai=True, project=PROJECT_ID, location="us-central1"
)

In [ ]:
from collections import defaultdict

def create_formatted_inputs(df):
    dialogues = defaultdict(list)

    # Group utterances by dialogue
    for _, row in df.iterrows():
        dialogues[row["Dialogue_ID"]].append((row["Speaker"], row["Utterance"], row["Emotion"]))

    # Convert to list format for easy processing
    dialogues = list(dialogues.values())

    # Generate formatted inputs with context
    formatted_inputs = []
    for dialogue in dialogues:
        context = ""
        for idx, (speaker, utterance, emotion) in enumerate(dialogue):
            context += f"{speaker}: {utterance}\n"

            last_speaker = speaker
            # Determine next speaker based on alternation if possible
            if idx + 1 < len(dialogue):
                next_speaker = dialogue[idx + 1][0]
            else:
                next_speaker = speaker  # Default to last speaker if no next available

            prompt = r"""### INSTRUCTIONS ###
Continue the conversation by generating **only the next line** spoken by the indicated character.
Your response must be empathetic, showing understanding or emotional attunement to the preceding dialogue.

### EXAMPLE ###

=== DIALOGUE HISTORY ===
Rachel: Hey!
Ross: Hi!
Rachel: What are you doing here?
Ross: Ah y'know, this building is on my paper route so I...
Rachel: Oh.
Ross: Hi.
Rachel: Hi.
Ross: How'd did it go?
Rachel: Oh well, the woman I interviewed with was pretty tough, but y'know thank God Mark coached me, because once I started talking about the fall line, she got all happy and wouldn't shut up.
Ross:

=== RESPONSE ===
That sounds like a huge relief.

### TASK ###

=== DIALOGUE HISTORY ===
{dialogue_hist}

=== RESPONSE ===
            """

            # full_input = (
            #     "### TASK ###\n"
            #     "Continue the conversation by generating **only one line** as the next speaker.\n"
            #     "This response should be **empathetic**, acknowledging or reflecting the emotional tone of the previous dialogue.\n"
            #     "DO NOT generate multiple lines.\n"
            #     "DO NOT summarize, analyze, or explain.\n"
            #     "Only generate one line and nothing more.\n\n"
            #     "### DIALOGUE HISTORY ###\n"
            #     f"{context.strip()}\n"
            #     f"{next_speaker}:"
            # )

            formatted_inputs.append(prompt.format(dialogue_hist=f"{context}{next_speaker}:"))

    return formatted_inputs

# Apply to each split
train_formatted_inputs = create_formatted_inputs(train_df)
dev_formatted_inputs = create_formatted_inputs(dev_df)
test_formatted_inputs = create_formatted_inputs(test_df)

print(f"Train inputs: {len(train_formatted_inputs)}")
print(f"Dev inputs: {len(dev_formatted_inputs)}")
print(f"Test inputs: {len(test_formatted_inputs)}")


Train inputs: 9989
Dev inputs: 1109
Test inputs: 2610


In [ ]:
print(train_formatted_inputs[11])

### INSTRUCTIONS ###
Continue the conversation by generating **only the next line** spoken by the indicated character.
Your response must be empathetic, showing understanding or emotional attunement to the preceding dialogue.

### EXAMPLE ###

=== DIALOGUE HISTORY ===
Rachel: Hey!
Ross: Hi!
Rachel: What are you doing here?
Ross: Ah y'know, this building is on my paper route so I...
Rachel: Oh.
Ross: Hi.
Rachel: Hi.
Ross: How'd did it go?
Rachel: Oh well, the woman I interviewed with was pretty tough, but y'know thank God Mark coached me, because once I started talking about the fall line, she got all happy and wouldn't shut up.
Ross:

=== RESPONSE ===
That sounds like a huge relief.

### TASK ###

=== DIALOGUE HISTORY ===
Chandler: also I was the point person on my company’s transition from the KL-5 to GR-6 system.
The Interviewer: You must’ve had your hands full.
Chandler: That I did. That I did.
The Interviewer: So let’s talk a little bit about your duties.
Chandler: My duties?  All 

In [ ]:
model = "gemini-2.0-flash-lite"

response = client.models.generate_content(
    model=model,
    contents=train_formatted_inputs[11],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.7
    )
)
print(response.text)

Oh, wow. That’s… that’s great to hear.



In [ ]:
import time, csv
from tqdm import tqdm
from typing import Any
from google.genai.types import GenerateContentConfig

# Your existing backoff method, updated to support config
def generate_with_backoff(prompt: str, config: GenerateContentConfig, max_retries: int = 5, backoff_factor: int = 2) -> str:
    attempt = 0
    while attempt < max_retries:
        try:
            response = client.models.generate_content(
                model="gemini-2.0-flash-lite",
                contents=[prompt],
                config=config
            )
            return response.text
        except Exception as e:
            attempt += 1
            delay = backoff_factor ** attempt
            print(f"Error encountered: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    raise Exception("Max retries reached. Failed to generate content.")

In [ ]:
config = GenerateContentConfig(
    max_output_tokens=500,
    temperature=0.7
)

def generate_and_save_responses(formatted_inputs, csv_filename):
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write the header
        writer.writerow(['Prompt', 'Response'])

        # Generate responses and write to CSV as they are generated
        for prompt in tqdm(formatted_inputs, desc=f"Generating responses for {csv_filename}"):
            response_text = generate_with_backoff(prompt, config=config)
            response_text_stripped = response_text.strip()

            # Write the prompt and the response to the CSV
            writer.writerow([prompt, response_text_stripped])

# Generate and save responses for train, dev, and test
generate_and_save_responses(train_formatted_inputs, 'train_targets.csv')
generate_and_save_responses(dev_formatted_inputs, 'dev_targets.csv')
generate_and_save_responses(test_formatted_inputs, 'test_targets.csv')

Generating responses for test_targets.csv: 100%|██████████| 2610/2610 [18:28<00:00,  2.35it/s]
